In [ ]:
from __future__ import annotations

from collections.abc import Iterable
from pathlib import Path

In [ ]:
type Position = tuple[int, int]

MAP: dict[Position, str] = {}

with Path("day12_example.txt").open() as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line):
            MAP[(row, col)] = char

In [ ]:
def same_neighbors(pos: Position) -> Iterable[Position]:
    """Find the up/down left/right neighbors of a position."""
    row, col = pos
    candidates = [(row, col - 1), (row, col + 1), (row - 1, col), (row + 1, col)]
    for candidate in candidates:
        if (candidate in MAP) and (MAP[candidate] == MAP[pos]):
            yield candidate

# Part 1
